# 使用backtrader实现一个基本策略的回测

In [3]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import os
import sys
from datetime import datetime
import pandas as pd
import akshare as ak
import backtrader as bt


In [4]:
df_stock = ak.stock_zh_a_spot_em()

## 1. 准备历史价格数据

In [5]:
# 以中国平安的近两年后复权日频数据为例
instrument_id = df_stock[df_stock['名称'] == '中国平安'].iloc[0]['代码']
df_price = ak.stock_zh_a_hist(symbol=instrument_id, period="daily", start_date="20201101", 
                              end_date='20221101', adjust="hfq")
df_price.head(5)

,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2020-11-02,176.47,173.93,177.09,173.49,638720,4.955636e+09,2.06,-0.69,-1.20,0.59
1,2020-11-03,175.01,177.79,178.15,174.27,615433,4.838929e+09,2.23,2.22,3.86,0.57
2,2020-11-04,177.03,178.47,179.17,176.67,434671,3.448542e+09,1.41,0.38,0.68,0.40
3,2020-11-05,180.17,181.35,182.83,179.87,537095,4.346337e+09,1.66,1.61,2.88,0.50
4,2020-11-06,181.67,182.07,182.97,180.51,510389,4.140944e+09,1.36,0.40,0.72,0.47


In [6]:
rename_map = {'日期': 'datetime', '开盘': 'open', '最高': 'high', '最低': 'low', '收盘': 'close', '成交量': 'volume'}
df_price.rename(columns=rename_map, inplace=True)
df_price = df_price[list(rename_map.values())].copy()
df_price['datetime'] = pd.to_datetime(df_price['datetime'])
df_price['volume'] = df_price['volume'].astype(int)

In [7]:
df_price.head(5)

,datetime,open,high,low,close,volume
0,2020-11-02,176.47,177.09,173.49,173.93,638720
1,2020-11-03,175.01,178.15,174.27,177.79,615433
2,2020-11-04,177.03,179.17,176.67,178.47,434671
3,2020-11-05,180.17,182.83,179.87,181.35,537095
4,2020-11-06,181.67,182.97,180.51,182.07,510389


## 2. 定义策略

实现一个很简单的均值回归策略：
1. 如果当前的价格低于过去20个交易日的价格的移动平均值，就买入，每次买入100股，如果本金不够，那么就能买多少买多少。
2. 如果当前价格高于过去20个交易日的价格的移动平均值，就卖出，每次卖出100股。

注意点：
1. 考虑佣金
2. 最大回撤不能超过50%。如果超过，那么直接平仓，结束策略。
3. 回测周期为2020.11.1 - 2022.11.1。

### 2.1 首先，先用一个简单的df进行测试

In [12]:
df_test = df_price.head(10).copy()

In [13]:
df_test

,datetime,open,high,low,close,volume
0,2020-11-02,176.47,177.09,173.49,173.93,638720
1,2020-11-03,175.01,178.15,174.27,177.79,615433
2,2020-11-04,177.03,179.17,176.67,178.47,434671
3,2020-11-05,180.17,182.83,179.87,181.35,537095
4,2020-11-06,181.67,182.97,180.51,182.07,510389
5,2020-11-09,184.47,186.47,184.07,185.43,809965
6,2020-11-10,187.95,188.97,185.57,185.87,789003
7,2020-11-11,185.87,187.53,184.77,186.51,542825
8,2020-11-12,186.97,188.41,183.07,184.01,490544
9,2020-11-13,181.63,182.87,178.73,180.15,707942


#### 2.2.1 关于执行顺序的归纳

以下测试策略，我们假设在奇数tick时买中国平安100股，在偶数tick时卖100股，不管其它任何的价格等信息，tick从索引1开始，索引1代表11月2日，规定每日挂的买单的tradeid为tick，次日的卖单的tradeid和今天的买单属于同一个交易，tradeid相同
<br><br> 
 - 正常模式的执行顺序如下：
1. 11月2日收盘后触发next，同时运行self.buy(size=100)
2. 11月3日使用11月3日的open挂11月2日的订单（tradeid=1），此时订单被Submitted & Accepted，同时订单成交Completed，触发notify_order
3. 11月3日挂的买单成交，导致tradeid=1的交易的仓位由0变为100，此时交易状态发生改变，触发notify_trade，此时trade.status == trade.Open
4. 11月3日收盘后触发next，同时运行self.sell(size=100)

5. 11月4日使用11月4日的open挂11月3日的订单（tradeid=1），此时订单被Submitted & Accepted，同时订单成交Completed，触发notify_order
6. 11月4日挂的卖单成交，导致tradeid=1的交易仓位由100变为0，此时交易状态发生改变，触发notify_trade，此时trade.status == trade.Closed
7. 11月4日收盘后触发next，同时运行self.buy(size=100)

8. 11月5日使用11月5日的open挂11月4日的订单（tradeid=3），此时订单被Submitted & Accepted，同时订单成交Completed，触发notify_order
9. 11月5日挂的买单成交，导致tradeid=3的交易仓位由0变为100，此时交易状态发生改变，触发notify_trade，此时trade.status == trade.Open
10. 11月5日收盘后触发next，同时运行self.sell(size=100)

11. 11月6日使用11月6日的open挂11月5日的订单（tradeid=3），此时订单被Submitted & Accepted，同时订单成交Completed，触发notify_order
12. 11月6日挂的卖单成交，导致tradeid=3的交易仓位由100变为0，此时交易状态发生改变，触发notify_trade，此时trade.status == trade.Closed
13. 11月6日收盘后触发next，同时运行self.buy(size=100)

14. ......

- coo模式的执行顺序如下：
1. 11月2日没有操作
2. 11月3日开盘前触发next_open，同时运行self.buy(size=100, tradeid=1)
3. 11月3日开盘后使用11月3日的open挂11月3日的订单（tradeid=1），同时订单被Submitted & Accepted，同时订单成交Completed，触发notify_order
4. 11月3日挂的买单成交，导致tradeid=1的交易的仓位由0变为100，此时交易状态发生改变，触发notify_trade，此时trade.status == trade.Open

5. 11月4日开盘前触发next_open，同时运行self.sell(size=100, tradeid=1)
6. 11月4日开盘后使用11月4日的open挂11月4日的订单（tradeid=1），同时订单被Submitted & Accepted，同时订单成交Completed，触发notify_order
7. 11月4日挂的卖单成交，导致tradeid=1的交易的仓位由100变为0，此时交易状态发生改变，触发notify_trade，此时trade.status == trade.Close

8. 11月5日开盘前触发next_open，同时运行self.buy(size=100, tradeid=3)
9. 11月5日开盘后使用11月5日的open挂11月5日的订单（tradeid=3），同时订单被Submitted & Accepted，同时订单成交Completed，触发notify_order
10. 11月5日挂的买单成交，导致tradeid=3的交易的仓位由0变为100，此时交易状态发生改变，触发notify_trade，此时trade.status == trade.Open

11. 11月6日开盘前触发next_open，同时运行self.sell(size=100, tradeid=3)
12. 11月6日开盘后使用11月6日的open挂11月6日的订单（tradeid=3），同时订单被Submitted & Accepted，同时订单成交Completed，触发notify_order
13. 11月6日挂的卖单成交，导致tradeid=3的交易的仓位由100变为0，此时交易状态发生改变，触发notify_trade，此时trade.status == trade.Close
14. ......

 - 正常模式下，在next里以索引0访问的当前bar的收盘价、移动平均值等都是基于今日的收盘价。

 - 每调用一次self.buy() or self.sell()，都会创建一笔订单，订单所属的trade、订单的orderid都可以自行指定。其中所属trade通过tradeid参数指定。orderid是自定义参数。其实在创建订单的时候也可以自己添加任意参数，这些参数在外部调用的时候，会被放在self.brokers.order[i].info这个dict中。在notify中调用的时候，会被放在order.info中。

In [14]:
class TestStrategy(bt.Strategy):
    
    s = '-' * 20
    
    def __init__(self):
        self.close = self.datas[0].close
        self.volume = self.datas[0].volume
        self.starting_value = self.broker.getvalue()
        # 记录是否作弊，注意它是如何取得cerebro参数的
        # cheat_on_open: 当日下单，当日开盘价成交
        self.coo = self.broker.p.coo
        self.coc = self.broker.p.coc
        
        # cheat_on_close: 当日下单，当日收盘价成交

        
    def log(self, content: str=None, dt: bool=True):
        if dt:
            time = self.datas[0].datetime.date(0)
            if content:
                print(f'{time}-- {content}')
            else:
                print(f'{time}-- ')
        else:
            if not content:
                return
            else:
                print(content)
    
    
    def notify_order(self, order):
        """
        每次订单状态发生改变时的回调函数。在每个tick，只要订单状态发生改变，就会调用notify_order，每个tick可能不止调用一次。
        e.g. 在11月2日下单的订单在11月3日才会被Submitted等。
        """
        
        if order.status == order.Submitted:  # 如果order的类型为Submitted，说明刚刚好到第二天开盘的时候，我们刚提交order
            self.log(f'\n{self.s}交易日{self.datas[0].datetime.date(0)}开盘{self.s}\n', dt=False)
        
        orderid = order.info['orderid']
        if order.isbuy():
            content = f'买单(orderid: {orderid}, 所属交易的tradeid: {order.tradeid}): '
        elif order.issell():
            content = f'卖单(orderid: {orderid}, 所属交易的tradeid: {order.tradeid})): '
        else:
            content = ''
        content += \
            '订单价格: {}, 执行价格: {}, 交易数量: {}, ' \
            '总交易额: {}, 订单类型: {}, 执行日期: {}' .format(order.price,
                                                           order.executed.price, 
                                                           order.executed.size, 
                                                           order.executed.value,
                                                           order.getstatusname(), 
                                                           self.datas[0].datetime.date(0))
        # bt.num2date(order.executed.dt)
        
        # self.log(self.bar_executed) 不存在这个属性！如果要使用的话需要自己定义
        self.log(content)
        
        if order.status == order.Completed:
            self.log(f'\n{self.s}交易日{self.datas[0].datetime.date(0)}收盘{self.s}\n\n\n', dt=False)   
#         if self.coo:  # 如果是coo模式，那么在每个交易日的开盘前调用next_open，在开盘的时候下单。当下单完成，说明
            
    
    def notify_trade(self, trade):
        """
        每次交易状态发生改变时的回调函数，只有仓位从0变为非0或者从非0变为0，才算是交易状态改变。
        注意，一个交易由多个的订单组成，我们通过在下单时规定tradeid来决定订单属于的交易。
        例如，self.sell(size=100, tradeid=1)和self.sell(size=200, tradeid=1)和self.buy(size=300, tradeid=1)属于
        同一个交易（tradeid=1的交易）。且这三个订单如果按照顺序下单，那么在self.sell(size=100, tradeid=1)时会触发notify_trade，
        此时trade.status == trade.Open；在self.buy(size=300, tradeid=1)时也会触发notify_trade，
        此时trade.status == trade.Open
        每个订单都可以理解为一个“子交易”，每个订单的交易状态改变都会触发一次notify_trade函数。
        
        """
        if trade.status == trade.Created:  
            # 一个交易被创建，但是仓位为0，此时虽然trade.status == trade.Created，但是根本不会调用notify_trade函数，
            # 因为仓位没变。
            self.log(f'Trade {trade.tradeid} created')
        elif trade.status == trade.Open:
            self.log(f'Trade {trade.tradeid} opened')
        elif trade.status == trade.Closed:
            self.log(f'Trade {trade.tradeid} closed')
            s = '毛收益 %f, 扣佣后收益 %f, 佣金 %f' % (trade.pnl, trade.pnlcomm, trade.commission)
            self.log(f'交易结束, 输出交易信息: {s}')
        else:
            self.log(trade.status)
            
#         if trade.isclosed:    
#             self.log(f'当前交易已经关闭')
#             s = '毛收益 %f, 扣佣后收益 %f, 佣金 %f' % (trade.pnl, trade.pnlcomm, trade.commission)
#             self.log(f'交易结束, 输出交易信息: {s}')
#         print(trade.Created)
            
        
    def start(self):
        """
        策略开始时的回调函数
        """
        
        self.log('STRATEGY BEGINS.', dt=False)
        if self.coo:
            self.log('THIS IS CHEAT ON OPEN MODE.', dt=False)
        elif self.coc:
            self.log('THIS IS CHEAT ON CLOSE MODE.', dt=False)
        else:
            self.log('THIS IS DEFAULT MODE.', dt=False)
            
        self.log(f'STARTING PORTFOLIO VALUE: {self.broker.get_value():.2f}\n\n', dt=False)
        
        
    def stop(self):
        """
        策略结束时的回调函数
        """
        self.close()
        self.order_info()
        
        self.log('STRATEGY ENDS.', dt=False)
        self.log(f'FINAL PORTFOLIO VALUE: {self.broker.get_value():.2f}', dt=False)
        
        
    def prenext(self):
        """
        在最小周期前的每个tick执行的函数
        """
        self.log('BEFORE THE MINIMUM PERIOD.')
    
    def notify_position(self):
        """
        自定义函数，输出持仓情况
        """
        if self.position:  # 如果持仓position的size不等于0的话，那么self.position为True
            self.log(f'当前持仓size不为0')
        else:
            self.log(f'当前持仓size为0')
            
        self.log(f'{self.position}', dt=False)
        # 现金 = 
        self.log(f'现金: {self.broker.get_cash()}')
        self.log(f'总持仓价值: {self.broker.get_value()}')
    
    def order_info(self):
        """
        自定义函数，打印出order的信息
        """
        self.log(f'{self.s}BEGIN OF ORDER INFO{self.s}', dt=False)
        for order in self.broker.orders:
            orderid = order.info['orderid']
            self.log(f'orderid: {orderid}, tradeid: {order.tradeid}, '
                     f'ref: {order.ref}, status: {order.getstatusname()}')
        self.log(f'{self.s}END OF ORDER INFO{self.s}', dt=False)            
        
    
    def place_order(self):
        if self.bar_executed % 2 != 0:
            self.buy(size=100, 
                     price=200, 
                     exectype=bt.Order.Limit, 
                     tradeid=self.bar_executed,  # tradeid是默认参数，可以在notify_trade中使用trade.tradeid访问
                     orderid=self.bar_executed) # 规定订单的orderid等于当前处理的行索引，存放在AutoOrderedDict中

            self.log(f'Created a buy order with orderid {self.bar_executed}, tradeid {self.bar_executed}')

        else:
            # 我们总是先买后卖，我们在每次卖出的时候规定tradeid为上一次买入的tradeid，使得上一次的买入交易平仓，
            # 即 trade.status == trade.Closed
            # orderid等参数都是自己定义的，会存放在order.info中。order.info是一个AutoOrderedDict（字典）
            self.sell(size=100, 
                      exectype=bt.Order.Market, 
                      tradeid=self.bar_executed - 1,  # tradeid是默认参数，可以在notify_trade中使用trade.tradeid访问
                      orderid=self.bar_executed) # orderid可以自己随便定义，可以重复，主要目的是区分订单，分类分析

            self.log(f'Created a sell order with orderid {self.bar_executed}, tradeid {self.bar_executed - 1}')
            
    def next(self):
        """
        除了coo模式以外，每个tick的回调函数，next函数在每个交易日结束后被调用。
        默认情况下，上一个交易日下的单会在这个交易日被submitted并且accepted，同时按照这个交易日的open价格进行买卖
        即：如果回测数据是日频的，那么上一个交易日下的单，在这个交易日才会产生订单状态的变化，才会调用notify_order函数输出log
        例如，使用self.sell()的日期是11月2日，那么notify_order的调用会在11月3日
        也就是说，订单下单时机是今日下午收盘后，订单执行时机是明日开盘，以开盘价成交（这是市价单的情况）。
        
        e.g. 执行顺序: 11月2日收盘后触发next_open，同时运行self.sell() or self.buy() -> 
                      11月3日使用11月3日的open挂11月2日的订单，触发notify_order -> 
                      11月3日收盘后触发next_open，同时运行self.sell() or self.buy() -> 
                      11月4日使用11月4日的open挂11月3日的单，触发notify_order
        """
# 这个注释掉的版本按照日期分隔
#         if not self.coo:
#             if len(self) == 1:
#                 self.log(f'\n{self.s}BEGIN OF TRADE DAY {self.datas[0].datetime.date(0)}{self.s}\n', dt=False)
#             self.bar_executed = len(self)
            
#             if self.bar_executed % 2 != 0:
#                 self.buy(size=100, price=200, exectype=bt.Order.Limit)  # 默认使用open作为卖一进行匹配
#                 self.log(f'Created a buy order.')
                
#             else:
#                 self.sell(size=100, exectype=bt.Order.Market)
#                 self.log(f'Created a sell order.')
            
#             self.order_info()
                
#             self.notify_position()

#             self.log(f'\n{self.s}END OF TRADE DAY {self.datas[0].datetime.date(0)}{self.s}\n\n\n', dt=False)
#             if len(self) < self.length:
#                 self.log(f'\n{self.s}BEGIN OF TRADE DAY {self.datas[0].datetime.date(1)}{self.s}', dt=False)

        # 按照最本质的逻辑分隔：next函数的调用在每次收盘之后
        if (not self.coc) and (not self.coo):
             
            self.log(
                f'\n{self.s}开始交易日{self.datas[0].datetime.date(0)}收盘之后的数据查看以及第二天的订单的下单{self.s}\n', 
                    dt=False)
            # 最后一天（11月13日）收盘后不下单，因为我们没有11月14日的数据。
            if len(self) < self.length:
                self.bar_executed = len(self)
                self.place_order()
            
            self.order_info()
                
            self.notify_position()

            self.log(f'\n{self.s}结束交易日{self.datas[0].datetime.date(0)}收盘之后的事宜{self.s}\n\n\n', dt=False)
        
        elif self.coc:
             
            self.log(
                f'\n{self.s}开始交易日{self.datas[0].datetime.date(0)}收盘之后的数据查看以及第二天的订单的下单{self.s}\n', 
                    dt=False)
            if len(self) < self.length:
                self.bar_executed = len(self)
                self.place_order()

            self.order_info()
                
            self.notify_position()

            self.log(f'\n{self.s}结束交易日{self.datas[0].datetime.date(0)}收盘之后的事宜{self.s}\n\n\n', dt=False)
            
        
        else:
            pass
            
    
    def next_open(self):
        """
        当cheat_on_open开启时候每个tick的回调函数，在每天的开盘前被调用。
        如果next函数和next_open函数均存在，即使在cheat_on_open模式下，也会优先运行next函数。
        在这个模式下，我们当日下单，并且使用当日开盘价成交（下单总是在next_open函数中）
        注意，在coo模式下，第一个交易日，即11月2日不运行策略，在11月3日下的单按照11月3日当天的开盘价格成交。之后
        
        
        当你在next_open函数中下单时，notify_order函数会在next_open函数之后执行
        
        执行顺序: 11月3日开盘前触发next_open， 同时运行self.sell() or self.buy()-> 
                 11月3日使用11月3日的open挂11月3日的单，触发notify_order -> 
                 11月4日开盘前触发next_open， 同时运行self.sell() or self.buy() -> 
                 11月4日使用11月4日的open挂11月4日的单，触发notify_order
        """
        if self.coo:
            
            self.log(f'\n{self.s}交易日{self.datas[0].datetime.date(0)}开盘前的数据查看以及下单{self.s}\n', dt=False)
            self.bar_executed = len(self)
            self.place_order()
                
            self.order_info()
                
            self.notify_position()
            
            
            self.log(f'\n{self.s}结束交易日{self.datas[0].datetime.date(0)}开盘前的事宜{self.s}\n\n\n', dt=False)


In [15]:
def backtrade(df, strategy):
    cerebro = bt.Cerebro()
    
#     cerebro = bt.Cerebro(cheat_on_open=True)
#     cerebro.broker.set_coc(True)
#     cerebro.broker.set_coo(True)
    
    # for log usage
    strategy.length = len(df)
    
    cerebro.addstrategy(strategy)
    
    data = bt.feeds.PandasData(dataname=df, datetime=-1)
    cerebro.adddata(data)
    
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.001)
    
    cerebro.run()

sell.order()的时间、next的时间、notify_order的时间

In [22]:
if __name__ == '__main__':
    backtrade(df_test, TestStrategy)

STRATEGY BEGINS.
THIS IS DEFAULT MODE.
STARTING PORTFOLIO VALUE: 100000.00



--------------------开始交易日2020-11-02收盘之后的数据查看以及第二天的订单的下单--------------------

2020-11-02-- Created a buy order with orderid 1, tradeid 1
--------------------BEGIN OF ORDER INFO--------------------
2020-11-02-- orderid: 1, tradeid: 1, ref: 55, status: Submitted
--------------------END OF ORDER INFO--------------------
2020-11-02-- 当前持仓size为0
--- Position Begin
- Size: 0
- Price: 0.0
- Price orig: 0.0
- Closed: 0
- Opened: 0
- Adjbase: None
--- Position End
2020-11-02-- 现金: 100000.0
2020-11-02-- 总持仓价值: 100000.0

--------------------结束交易日2020-11-02收盘之后的事宜--------------------




--------------------交易日2020-11-03开盘--------------------

2020-11-03-- 买单(orderid: 1, 所属交易的tradeid: 1): 订单价格: 200, 执行价格: 0.0, 交易数量: 0, 总交易额: 0.0, 订单类型: Submitted, 执行日期: 2020-11-03
2020-11-03-- 买单(orderid: 1, 所属交易的tradeid: 1): 订单价格: 200, 执行价格: 0.0, 交易数量: 0, 总交易额: 0.0, 订单类型: Accepted, 执行日期: 2020-11-03
2020-11-03-- 买单(orderid: 1, 所属交易的tradeid

In [11]:
df_test

NameError: name 'df_test' is not defined

在Backtrader中，`self.buy`函数有以下参数：

- `data`: 数据对象。默认值为`None`。
- `size`: 交易量。默认值为`None`。
- `price`: 价格。默认值为`None`。
- `plimit`: 限价单价格。默认值为`None`。
- `exectype`: 执行类型。默认值为`bt.Order.Market`。
- `valid`: 订单有效期。默认值为`None`。
- `tradeid`: 交易ID。默认值为`None`。
- `oca`: OCA组ID。默认值为`None`。
- `name`: 订单名称。默认值为`None`。
- `parent`: 父订单。默认值为`None`。
- `transmit`: 是否传输订单。默认值为`True`。
- `trailamount`: 跟踪止损金额。默认值为`None`。
- `trailpercent`: 跟踪止损百分比。默认值为`None`。
- `args`: 附加参数。默认值为`None`。
- `kwargs`: 附加关键字参数。默认值为`None`。

In [45]:
class MyStrategy(bt.Strategy):
    def next(self):
        order = self.buy(size=1, exectype=bt.Order.Market, ref=1)
        print('Order ref:', order.ref)

    def notify_order(self, order):
        print('Order ref:', order.ref)
